# Cluster map

In [1]:
%matplotlib inline

import os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import pyarrow

In [3]:
# Verify SparkContext
print(sc)

# Print Spark version
print(sc.version)

<SparkContext master=mesos://vm-75063.lal.in2p3.fr:5050 appName=PySparkShell>
2.4.4


In [4]:
from pyspark.sql.functions import col, pandas_udf 
from pyspark.sql.types import LongType

from pyspark.sql.types import IntegerType,FloatType
from pyspark.sql.functions import pandas_udf, PandasUDFType

from pyspark.sql import functions as F
from pyspark.sql.functions import randn


In [5]:
# here is how we create a function ('Ang2Pix') that can be called by dataframes
# it takes as input the "ra" and "dec" values (which are not very different from theta/phi)
# and returns the pixel number (but as pandas series for efficiency)
import numpy as np
import pandas as pd
import healpy as hp

nside=512

In [6]:
! hadoop dfs -ls /lsst/DC2/cosmoDC2

DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

Found 5 items
drwxr-xr-x   - stephane.plaszczynski lsst          0 2020-12-11 15:12 /lsst/DC2/cosmoDC2/cosmoDC2_v1.1.4_image
drwxr-xr-x   - stephane.plaszczynski lsst          0 2019-10-16 10:52 /lsst/DC2/cosmoDC2/cosmoDC2_v1.1.4_image_double.parquet
drwxr-xr-x   - stephane.plaszczynski lsst          0 2020-04-23 17:28 /lsst/DC2/cosmoDC2/cosmoDC2_v1.1.4_image_nofaint.parquet
drwxr-xr-x   - stephane.plaszczynski lsst          0 2020-09-07 19:17 /lsst/DC2/cosmoDC2/shear_cosmoDC2
drwxr-xr-x   - stephane.plaszczynski lsst          0 2019-04-16 18:40 /lsst/DC2/cosmoDC2/xyz_v1.1.4_hive


In [31]:
! hadoop dfs -ls /lsst/DC2

DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

Found 20 items
drwxr-xr-x   - stephane.plaszczynski lsst           0 2020-06-12 10:23 /lsst/DC2/DR6axCdc2.parquet
drwxr-xr-x   - stephane.plaszczynski lsst           0 2020-12-11 12:11 /lsst/DC2/cosmoDC2
-rw-r--r--   3 stephane.plaszczynski lsst 83486737444 2020-05-28 19:33 /lsst/DC2/dc2_object_run2.2i_dr3.parquet
-rw-r--r--   3 stephane.plaszczynski lsst 44384598496 2020-06-05 13:15 /lsst/DC2/dc2_object_run2.2i_dr6.parquet
drwxr-xr-x   - stephane.plaszczynski lsst           0 2020-10-07 20:49 /lsst/DC2/dc2_object_run2.2i_dr6_wfd_v1
drwxr-xr-x   - stephane.plaszczynski lsst           0 2020-06-26 10:36 /lsst/DC2/dc2_object_run2.2i_dr6b
drwxr-xr-x   - stephane.plaszczynski lsst           0 2020-07-28 18:46 /lsst/DC2/dc2_object_run2.2i_dr6c
drwxr-xr-x   - stephane.plaszczynski lsst           0 2020-09-07 11:52 /lsst/DC2/dc2_object_run2.2i_dr6d
drwxr-xr-x   - stephane.plaszczynski l

In [7]:
#base_dir="/lsst/DC2/cosmoDC2/shear_cosmoDC2"

In [11]:
#fullfilename="/lsst/DC2/cosmoDC2/cosmoDC2_v1.1.4_image.parquet"
fullfilename="/lsst/DC2/cosmoDC2/cosmoDC2_v1.1.4_image"
datafile=fullfilename
FLAG_ONE_FILE_INPUT=True

In [9]:
#from subprocess import Popen, PIPE
#hdfs_path = base_dir
#process = Popen(f'hdfs dfs -ls -h {hdfs_path}', shell=True, stdout=PIPE, stderr=PIPE)
#std_out, std_err = process.communicate()
#list_of_file_names = [fn.split(' ')[-1].split('/')[-1] for fn in std_out.decode().readlines()[1:]][:-1]
#list_of_file_names_with_full_address = [fn.split(' ')[-1] for fn in std_out.decode().readlines()[1:]][:-1]

In [10]:
#for fn in std_out.readlines():
#    print(fn)

In [12]:
from pyspark.sql import SparkSession

# Initialise our Spark session
spark = SparkSession.builder.getOrCreate()

In [13]:
# Read the data as DataFrame
if FLAG_ONE_FILE_INPUT:
    df = spark.read.format("parquet").load(datafile)
else: # read all files
    #the_list_of_files=os.listdir(base_dir)
    #the_sorted_list_of_files=sorted(the_list_of_files)
    #print("sorted files : ",the_sorted_list_of_files[:5])
    #selected_files=[]
    #for filename in the_sorted_list_of_files:
    #    sel_filename=re.findall("^dc2_object_run2.2i_dr6c_tract[0-9][0-9][0-9][0-9].parquet$",filename)
    #    if len(sel_filename) > 0:
    #        selected_files.append(sel_filename[0])
    #print("selected files : ", selected_files[:5])
    #InputPath = [base_dir + "/{}".format(filename) for filename in selected_files]
    InputPath = all_files  # filename from the decoded index file
    df = spark.read.parquet(*InputPath)        

In [14]:
df.printSchema()

root
 |-- sed_2998_186_disk: float (nullable = true)
 |-- sed_17402_2596_disk: float (nullable = true)
 |-- sed_15143_2259_bulge: float (nullable = true)
 |-- Mag_true_y_lsst_z0: float (nullable = true)
 |-- sed_8329_517_bulge: float (nullable = true)
 |-- mag_i_lsst: double (nullable = true)
 |-- sed_13177_1966_disk: float (nullable = true)
 |-- mag_true_z_sdss: float (nullable = true)
 |-- sed_2407_591_no_host_extinction: float (nullable = true)
 |-- sed_1933_474_disk_no_host_extinction: float (nullable = true)
 |-- sed_3590_222: float (nullable = true)
 |-- sed_4048_251: float (nullable = true)
 |-- sed_8329_517: float (nullable = true)
 |-- mag_u_lsst_no_host_extinction: double (nullable = true)
 |-- velocity_x: double (nullable = true)
 |-- sed_1246_306_bulge: float (nullable = true)
 |-- sed_17402_2596_no_host_extinction: float (nullable = true)
 |-- sed_1552_381: float (nullable = true)
 |-- Mag_true_u_lsst_z0_no_host_extinction: float (nullable = true)
 |-- sed_6548_406_disk_no

In [15]:
data=df.select("halo_id","ra","dec","position_x","position_y","position_z","redshift","stellar_mass","is_central")

In [17]:
data.show()

+----------+------------------+-------------------+------------------+------------------+-------------------+--------------------+-------------+----------+
|   halo_id|                ra|                dec|        position_x|        position_y|         position_z|            redshift| stellar_mass|is_central|
+----------+------------------+-------------------+------------------+------------------+-------------------+--------------------+-------------+----------+
|    151487| 49.31259353683868|  -40.2957920342933|  45.5167958434199|52.941690364354095| -59.20144739285322|0.021649390521782053|1.61537874E10|      true|
|    151487| 49.47752863847466| -40.27953751237621| 45.42154661366637| 53.13958181461818|-59.242194807025754|0.021912149530828096| 1.45056048E8|     false|
| 100151487| 51.23465325073303| -40.12060710649598| 43.54176857102085| 54.22216066172425| -58.60158356142716|0.021491185880432306| 3.39115392E8|      true|
| 200151487| 49.46696774834902| -39.98940095061212| 45.164554219

In [22]:
data.count()

324194504

In [17]:
#data.describe().show()

In [23]:
center_clu=data.filter(data.is_central==True)

In [24]:
center_clu.count()

241445447

In [25]:
center_clu.show()

+----------+------------------+-------------------+------------------+------------------+-------------------+--------------------+-------------+----------+
|   halo_id|                ra|                dec|        position_x|        position_y|         position_z|            redshift| stellar_mass|is_central|
+----------+------------------+-------------------+------------------+------------------+-------------------+--------------------+-------------+----------+
|    151487| 49.31259353683868|  -40.2957920342933|  45.5167958434199|52.941690364354095| -59.20144739285322|0.021649390521782053|1.61537874E10|      true|
| 100151487| 51.23465325073303| -40.12060710649598| 43.54176857102085| 54.22216066172425| -58.60158356142716|0.021491185880432306| 3.39115392E8|      true|
| 200151487| 49.46696774834902| -39.98940095061212| 45.16455421985035|52.819206345249235| -58.29223310443717|0.021335146372658986| 2.54158784E8|      true|
| 300151487|50.803636542046334|-40.039271615821946|12.9113224190

In [28]:
from pyspark.ml.feature import Bucketizer

In [29]:
minmax = center_clu.agg(F.min("stellar_mass"), F.max("stellar_mass")).head()

In [30]:
minmax

Row(min(stellar_mass)=21214.576171875, max(stellar_mass)=4468432699392.0)